In [61]:
from sklearn.ensemble import RandomForestRegressor
import os
import json
import pickle
from collections import defaultdict
from collections import Counter
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import gc
from sklearn.inspection import permutation_importance
# training and permutation importance code taken from sklearn

In [62]:
with open('mappings/RF/data_dict2.pkl', 'rb') as fp:
    data_dict = pickle.load(fp)

In [63]:
data_dict[962001]

[1.9835831548893639,
 0.43318300041595537,
 93.65003950517811,
 143,
 29174668,
 204018.65734265733,
 40,
 25,
 1]

In [64]:
data = list(data_dict.items())
data[:3]

[(0,
  [1.9782298358315489,
   0.25206242699126885,
   87.25048873203248,
   52,
   11532858,
   221785.73076923078,
   47,
   50,
   1]),
 (1,
  [1.9825124910778014,
   0.22576916274870246,
   26.311126242565063,
   39,
   11668180,
   299184.10256410256,
   23,
   27,
   1]),
 (2,
  [1.9950035688793717,
   0.5280443966756907,
   5.880931026409286,
   64,
   14033889,
   219279.515625,
   51,
   46,
   1])]

In [65]:
dall = np.array([v for k,v in data_dict.items()])


In [66]:
data.sort(key = lambda x: x[1][-1], reverse=True)

In [67]:
data[:3]

[(768280,
  [1.9967880085653102,
   0.45777324817692555,
   29.58511978830257,
   13,
   3210659,
   246973.76923076922,
   12,
   15,
   15123]),
 (25495,
  [1.9825124910778005,
   0.34816514325305115,
   241.2848982944465,
   115,
   27640708,
   240353.98260869566,
   21,
   40,
   14812]),
 (667135,
  [1.9878658101356175,
   0.4199799877958231,
   3.7162546450329983,
   227,
   47214858,
   207994.9691629956,
   28,
   42,
   10911])]

In [68]:
top_k = 10

In [69]:
top = data[:top_k]
top_ids = set([x[0] for x in top])
top_ids

{25495, 48201, 452505, 468318, 667135, 691217, 739360, 742759, 768280, 966695}

In [70]:
# Format
# name, followers, L1, derived popularity, #tracks, total_duration, avg_sl, albums, total_author_count 

In [71]:
# # get names
# # IDEA: can also look at playlist_num['num_albums']
# MPD = 'mpd/'
# playlist_files = [os.path.join(MPD, x) for x in os.listdir(MPD)]
# top_data = {}
# for file in playlist_files:
#     with open(file, 'rb') as fp:
#         data = json.load(fp)
#     for playlist in data['playlists']:
#         ID = playlist['pid']
#         if ID in top_ids:
#             top_data[ID] = playlist['name']

In [72]:
top_data

{691217: 'The weekend',
 768280: 'FARRUKO',
 667135: 'Christmas Instrumental',
 565974: 'almost famous',
 966695: 'Pirates Of The Caribbean',
 468318: 'REGEATON',
 48201: 'Summer 16',
 452505: "90's kid",
 742759: 'Yoga',
 25495: 'Wiz Khalifa',
 990562: 'calm music',
 935821: 'Tercer cielo',
 54489: 'Pop Country',
 739360: 'classic rock',
 415171: 'Ellie Goulding'}

In [73]:
print(np.min(dall, axis=1), np.max(dall, axis=1))

[0.25206243 0.22576916 0.5280444  ... 0.26472305 0.36368698 0.24645816] [11532858. 11668180. 14033889. ...  5934551.  8502131.  6016033.]


In [85]:
# name, followers, L1, derived popularity, #tracks, total_duration, avg_sl, albums, total_author_count 
for playlist in top:
    ID = playlist[0]
    vals = playlist[1].copy()
    del vals[1]
    name = top_data[ID]
    vals.insert(0, vals[-1])
    del vals[-1]
    vals = np.array(vals)
    vals[4] *= 1.66667e-5
    vals[5] *= 1.66667e-5
    print(name, ' &', int(vals[0]), ' & ', ' & '.join([str(x) for x in np.around(vals[1:], 3)]), '\\\ \n \hline')


FARRUKO  & 15123  &  1.997 & 29.585 & 13.0 & 53.511 & 4.116 & 12.0 & 15.0 \\ 
 \hline
Wiz Khalifa  & 14812  &  1.983 & 241.285 & 115.0 & 460.679 & 4.006 & 21.0 & 40.0 \\ 
 \hline
Christmas Instrumental  & 10911  &  1.988 & 3.716 & 227.0 & 786.916 & 3.467 & 28.0 & 42.0 \\ 
 \hline
Yoga  & 9345  &  1.999 & 0.221 & 88.0 & 238.672 & 2.712 & 8.0 & 7.0 \\ 
 \hline
Summer 16  & 6568  &  1.974 & 28.535 & 24.0 & 91.635 & 3.818 & 23.0 & 41.0 \\ 
 \hline
The weekend  & 6337  &  1.989 & 357.709 & 65.0 & 323.438 & 4.976 & 13.0 & 16.0 \\ 
 \hline
REGEATON  & 6326  &  1.972 & 32.824 & 227.0 & 845.541 & 3.725 & 156.0 & 122.0 \\ 
 \hline
90's kid  & 5822  &  1.976 & 20.594 & 86.0 & 349.823 & 4.068 & 77.0 & 70.0 \\ 
 \hline
Pirates Of The Caribbean  & 5787  &  1.989 & 21.021 & 58.0 & 235.464 & 4.06 & 5.0 & 14.0 \\ 
 \hline
classic rock  & 5164  &  1.966 & 30.017 & 211.0 & 1043.757 & 4.947 & 163.0 & 91.0 \\ 
 \hline


In [86]:
#   followers, L1, derived popularity, #tracks, total_duration, avg_sl, albums, total_author_count
# min, max, mean, mode

In [96]:

for fn in ['min', 'max', 'mode', 'mean']:
    print('\\ \n \hline\n', end='')
    print(fn, ' & ', end='')
    for ix in [-1,0,2,3,4,5,6,7]:
        if fn == 'min':
            val = np.around(np.min(dall[:,ix]),3)
        elif fn == 'max':
             val = np.around(np.max(dall[:,ix]),3)
        elif fn == 'mode':
             val = np.around(stats.mode(list(dall[:,ix])),3)[0][0]
        elif fn == 'mean':
             val = np.around(np.mean(dall[:,ix]),3)
        if ix == 4 or ix == 5:
            print(np.around(val*1.66667e-5,3), '&', end='')
        else:
            print(val, ' &', end='')

\ 
 \hline
min  & 1.0  &1.905  &0.002  &5.0  &1.678 &0.336 &2.0  &1.0  &\ 
 \hline
max  & 15123.0  &2.0  &989.932  &250.0  &2572.402 &61.682 &217.0  &319.0  &\ 
 \hline
mode  & 1.0  &1.987  &15.053  &15.0  &44.363 &3.697 &15.0  &14.0  &\ 
 \hline
mean  & 1.982  &1.983  &87.565  &44.218  &172.702 &3.915 &31.317  &32.785  &